In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [4]:
data_dir = os.path.join('..','..','data')

with open(os.path.join(data_dir,'maps','geojson','Nakhon-Si-Thammarat-choropleth-dengue-season.geojson')) as f:
    shapefile = json.load(f)
shapefile['features'][0]['properties']

{'AP_EN': 'Phrom Khiri',
 'AP_IDN': '8002',
 'AP_TN': 'พรหมคีรี',
 'Area': 1070,
 'Avg_BI': 20,
 'BI_dengue_season': 0,
 'BI_entire_year': 33.5,
 'Breeding_site_classes': {'bin': 1,
  'bowl': 0,
  'bucket': 314,
  'cup': 37,
  'jar': 177,
  'pottedplant': 147,
  'tire': 2,
  'vase': 0},
 'Breeding_site_counts': 678,
 'Breteau_index': {'2011': 0,
  '2012': 0,
  '2013': 0,
  '2014': 50,
  '2015': 33,
  '2016': 0,
  '2017': 0},
 'Dengue_cases': 25,
 'Density': 6,
 'Density_population': 8,
 'PV_EN': 'Nakhon Si Thammarat',
 'PV_IDN': '80',
 'PV_TN': 'นครศรีธรรมราช',
 'Population': 8387,
 'TB_EN': 'Thon Hong',
 'TB_IDN': '800204',
 'TB_TN': 'ทอนหงส์'}

In [8]:
shapefile['features'][0]['properties']

{'AP_EN': 'Phrom Khiri',
 'AP_IDN': '8002',
 'AP_TN': 'พรหมคีรี',
 'Area': 1070,
 'Avg_BI': 20,
 'BI_dengue_season': 0,
 'BI_entire_year': 33.5,
 'Breeding_site_classes': {'bin': 147,
  'bowl': 1,
  'bucket': 314,
  'cup': 2,
  'jar': 0,
  'pottedplant': 177,
  'tire': 37,
  'vase': 0},
 'Breeding_site_counts': 678,
 'Breteau_index': {'2011': 0,
  '2012': 0,
  '2013': 0,
  '2014': 50,
  '2015': 33,
  '2016': 0,
  '2017': 0},
 'Dengue_cases': 25,
 'Density': 6,
 'Density_population': 8,
 'PV_EN': 'Nakhon Si Thammarat',
 'PV_IDN': '80',
 'PV_TN': 'นครศรีธรรมราช',
 'Population': 8387,
 'TB_EN': 'Thon Hong',
 'TB_IDN': '800204',
 'TB_TN': 'ทอนหงส์'}

In [5]:
df_detect = pd.read_csv(os.path.join(data_dir,'breeding-sites','csv','addrcode-index','detection.csv')) 
df_detect['date'] = pd.to_datetime(df_detect['date'], format='%Y-%m')
df_detect['addrcode'] = df_detect['addrcode'].astype('int')
df_detect = df_detect.set_index('date')
df_detect = df_detect.sort_index()
df_detect.head()

,addrcode,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,
2011-12-01,103802,270,15,829,9,71,2246,165,4,3609
2011-12-01,104502,153,17,523,10,21,850,135,6,1715
2011-12-01,104401,349,47,2160,29,181,4844,409,12,8031
2011-12-01,103401,691,78,2572,64,107,4684,475,19,8690
2011-12-01,103801,623,69,2278,65,120,5411,445,23,9034


In [6]:
x = []
cnt = 0
for feature in shapefile['features']:
    
    prop = feature['properties']
    
    addrcode = prop['TB_IDN']
    
    brd_site_classes = df_detect.loc[df_detect['addrcode'] == int(addrcode)][['bin','bowl','bucket','cup','jar','pottedplant','tire','vase']]
    
    if len(brd_site_classes) == 0:
        feature['properties']['Breeding_site_classes'] = {
            'bin': 0,
            'bowl': 0,
            'bucket': 0,
            'cup':0,
            'jar': 0,
            'pottedplant': 0,
            'tire': 0,
            'vase':0
        }
    else:
        feature['properties']['Breeding_site_classes'] = {
            'bin': int(brd_site_classes.bin.values[0]),
            'bowl': int(brd_site_classes.bowl.values[0]),
            'bucket': int(brd_site_classes.bucket.values[0]),
            'cup': int(brd_site_classes.cup.values[0]),
            'jar': int(brd_site_classes.jar.values[0]),
            'pottedplant': int(brd_site_classes.pottedplant.values[0]),
            'tire': int(brd_site_classes.tire.values[0]),
            'vase': int(brd_site_classes.vase.values[0]),
        }

In [7]:
with open(os.path.join(data_dir, 'maps', 'geojson', 'Nakhon-Si-Thammarat-choropleth-dengue-season-2.geojson'), 'w') as FILE:
    json.dump(shapefile, FILE, indent=4, ensure_ascii=False, sort_keys=True)